In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [2]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float', 
'habitaciones': 'float', 
'garages': 'float', 
'banos': 'float', 
'metroscubiertos': 'float', 
'metrostotales': 'float', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float', 
'usosmultiples': 'float', 
'piscina': 'float', 
'escuelascercanas': 'float', 
'centroscomercialescercanos': 'float', 
'precio': 'float', }

data = pd.read_csv("train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


# Pre-procesamiento de data

In [3]:
data["ciudad"].nunique()

875

In [4]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [5]:
data.isnull().sum().sum()

535127

In [6]:
data.size

5520000

In [7]:
(data.isnull().sum().sum()/(data.size))*100

9.694329710144928

Los Nulls representan el 10% de los datos.

# CON XGBOOST NO HACE FALTA PREOCUPARSE POR LOS NULLs

# Levanto el csv de test para calcularle los features en paralelo

In [8]:
test = pd.read_csv("test.csv", dtype = dtypes)
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


In [9]:
len(test)

60000

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [10]:
train_set = pd.DataFrame()

In [11]:
test_set = pd.DataFrame()

# Agrego las columnas que ya se encuentran

In [12]:
data.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [13]:
train_set = data.loc[:, ['id','antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio']]

In [14]:
train_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,19.310205,-99.227655,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,19.301890,-99.688015,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1150000.0


### Lo mismo para test

In [15]:
test_set = test.loc[:, ['id', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos']]

In [16]:
test_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,29.0,3.0,NaN,4.0,300.0,NaN,NaN,19.408668,-99.246767,0.0,0.0,0.0,0.0,0.0
1,51775,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,21.032480,-89.592424,0.0,0.0,0.0,0.0,0.0
2,115253,0.0,2.0,1.0,2.0,87.0,100.0,23620.0,19.332829,-99.152913,0.0,0.0,0.0,0.0,1.0
3,299321,2.0,2.0,2.0,2.0,86.0,86.0,129347.0,16.860487,-99.878383,0.0,0.0,0.0,0.0,0.0
4,173570,10.0,2.0,1.0,1.0,80.0,76.0,57125.0,19.640482,-99.127273,0.0,0.0,0.0,1.0,1.0


# Nuevos features

## OJO!!! Pensar muy bien si tiene sentido agregar ese feature o no... Porque agregar de más puede ser muy malo a priori. Lo más simple suele ser lo mejor!!!

## Fecha de publicación

In [17]:
data['fecha'] = pd.to_datetime(data['fecha'])
data['anio'] = data['fecha'].dt.year

In [18]:
data['anio'].value_counts()

2016    94038
2015    51470
2014    40572
2013    30386
2012    23534
Name: anio, dtype: int64

In [19]:
train_set['anio_publ'] = data['anio']

In [20]:
test_set['anio_publ'] = pd.to_datetime(test['fecha']).dt.year

### Feature total de antiguedad por anio

In [21]:
data.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239990,239991,239992,239993,239994,239995,239996,239997,239998,239999
id,254099,53461,247984,209067,185997,126147,139233,5013,44962,134537,...,87498,137337,54886,207892,110268,119879,259178,131932,146867,121958
titulo,depto. tipo a-402,condominio horizontal en venta,casa en venta urbi 3 recamaras tonala,casa sola en toluca zinacantepec con credito i...,paseos del sol,departamento en venta taxqueña,de oportunidad casa en san lorenzo,casa emilia en venta en selvamar playa del carmen,pre- venta preciosos depas 2 recamaras con sub...,terreno,...,casa en venta: bosques del contry,departamento residencial coyuya,casa en venta,bugambilias (ciudad),hermosa casa en villa de los belenes,bonita casas de 2 recamaras a 10 minutos del c...,casa en condominio a 10 min. del centro de toluca,nicolas san juan,casa sola. javier rojo gomez.,departamento en bosques de las lomas / av. st...
descripcion,"depto. interior de 80.15m2, consta de sala com...","<p>entre sonora y guerrero, atr&aacute;s del h...",descripcion \nla mejor ubicacion residencial e...,casa en privada con caseta de vigilancia casas...,bonito departamento en excelentes condiciones ...,"amplio departamento, estancia de sala y comedo...","ubicada en esquina, pertenece san lorenzo agen...",casa emilia en venta playa del carmenfracciona...,<p>pre-venta de preciosos departamento ecologi...,"terreno de 5.500m2 bardeado, uso de suelo h-20...",...,"<p>casa en venta, en magníficas condiciones; e...","departamento ubicado en planta baja, con excel...",bonita casa para remodelar en una calle cerrad...,coto privado de tan solo 7 casas donde cada fa...,"<p>moderna casa 3 pisos, muro llor&oacute;n , ...",vendo casa en bosques de ica residencial a 10 ...,"casa con un jardin amplio, un cuarto de servic...","departamento con excelente ubicación, muy cerc...","casa sola, dividida en cuatro departamentos de...","id:19816, muy bonito e iluminado departamento,..."
tipodepropiedad,Apartamento,Casa en condominio,Casa,Casa,Apartamento,Apartamento,Casa,Casa,Apartamento,Terreno,...,Casa,Apartamento,Casa en condominio,Casa,Casa,Casa,Casa,Apartamento,Casa,Apartamento
direccion,Avenida Division del Norte 2005,AV. MEXICO,Urbi Tonala,IGNACIO MANUEL ALTAMIRANO 128,PASEOS DEL SOL,Condominio Tlalpan 2B,NaN,condominio el trebol,BUENAVISTA DEPTOS CON SUBSIDIO,Av. Morelos,...,NaN,Coyuya 200,Cerrada villa Picadilly,Paseo de la Cañada,"MANUEL AMAYA,ENTRE SEBASTIAN ALLENDE Y AMADO A...",BOSQUES,Filiberto Navas 325,Nicolas San Juan,Javier Rojo Gomez 120,AVE. STIM
ciudad,Benito Juárez,La Magdalena Contreras,Tonalá,Zinacantepec,Zapopan,Coyoacán,Oaxaca de Juárez,Playa del Carmen,Villa de Alvarez,Ixtapaluca,...,Guadalupe,Iztacalco,Huixquilucan,Zapopan,Zapopan,Zinacantepec,Toluca,Benito Juárez,Iztapalapa,Cuajimalpa de Morelos
provincia,Distrito Federal,Distrito Federal,Jalisco,Edo. de México,Jalisco,Distrito Federal,Oaxaca,Quintana Roo,Colima,Edo. de México,...,Nuevo León,Distrito Federal,Edo. de México,Jalisco,Jalisco,Edo. de México,Edo. de México,Distrito Federal,Distrito Federal,Distrito Federal
antiguedad,NaN,10,5,1,10,5,NaN,2,1,NaN,...,20,20,10,1,3,0,0,20,20,1
habitaciones,2,3,3,2,2,2,3,4,2,NaN,...,3,2,3,3,2,2,3,2,4,3
garages,1,2,2,1,1,1,1,2,1,NaN,...,2,1,NaN,2,2,2,3,1,0,2


In [22]:
ant = data
ant['count'] = 1
ant = ant.groupby(by='antiguedad').sum()[['count','habitaciones','id']]
#ant.sort_values(ascending=False).tail(5)
ant = ant.reset_index()
ant.head()

,antiguedad,count,habitaciones,id
0,0.0,50335,126567.0,7.521382e+09
1,1.0,12353,32263.0,1.857008e+09
2,2.0,5059,13537.0,7.697436e+08
3,3.0,5616,15068.0,8.434886e+08
4,4.0,7944,21356.0,1.196237e+09


In [23]:
def hab_ant(row):
    #if(row['antiguedad'] <= 10 ):
        return row['count'] * 1
    #return row['habitaciones'] * 0.5

In [24]:
ant2 = ant
ant2['c_hab_ant'] = ant2.apply(hab_ant,axis=1)
ant2.head(15)

,antiguedad,count,habitaciones,id,c_hab_ant
0,0.0,50335,126567.0,7.521382e+09,50335.0
1,1.0,12353,32263.0,1.857008e+09,12353.0
2,2.0,5059,13537.0,7.697436e+08,5059.0
3,3.0,5616,15068.0,8.434886e+08,5616.0
4,4.0,7944,21356.0,1.196237e+09,7944.0
5,5.0,33268,89658.0,4.970856e+09,33268.0
6,6.0,3169,8520.0,4.761291e+08,3169.0
7,7.0,2046,5513.0,3.130939e+08,2046.0
8,8.0,2418,6760.0,3.712067e+08,2418.0
9,9.0,1065,2905.0,1.624691e+08,1065.0


In [25]:
data = data.drop(columns='count')

In [26]:
ant3 = ant2[['c_hab_ant','antiguedad']]
ant3.head()

,c_hab_ant,antiguedad
0,50335.0,0.0
1,12353.0,1.0
2,5059.0,2.0
3,5616.0,3.0
4,7944.0,4.0


In [27]:
train_set = train_set.merge(ant3,on = 'antiguedad',how = 'left')
train_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239990,239991,239992,239993,239994,239995,239996,239997,239998,239999
id,254099,53461,247984,209067,185997,126147,139233,5013,44962,134537,...,87498,137337,54886,207892,110268,119879,259178,131932,146867,121958
antiguedad,NaN,10,5,1,10,5,NaN,2,1,NaN,...,20,20,10,1,3,0,0,20,20,1
habitaciones,2,3,3,2,2,2,3,4,2,NaN,...,3,2,3,3,2,2,3,2,4,3
garages,1,2,2,1,1,1,1,2,1,NaN,...,2,1,NaN,2,2,2,3,1,0,2
banos,2,2,2,1,1,1,2,3,1,NaN,...,1,1,3,3,2,1,3,2,4,2
metroscubiertos,80,268,144,63,95,75,140,293,58,250,...,118,68,270,150,160,67,200,138,235,149
metrostotales,80,180,166,67,95,90,160,293,NaN,NaN,...,124,68,293,150,90,NaN,250,138,137,NaN
idzona,23533.0,24514.0,48551.0,53666.0,47835.0,23650.0,73510.0,130510.0,9010.0,59171.0,...,72083.0,24118.0,55589.0,48046.0,48029.0,53666.0,51954.0,50003995.0,24162.0,23750.0
lat,NaN,19.3102,NaN,19.3019,NaN,19.3006,17.1435,20.6726,NaN,19.316,...,25.6612,19.4033,NaN,20.6156,20.6045,NaN,19.2947,NaN,19.3667,NaN
lng,NaN,-99.2277,NaN,-99.688,NaN,-99.1485,-96.8035,-87.038,NaN,-98.887,...,-100.251,-99.119,NaN,-103.456,-103.45,NaN,-99.6929,NaN,-99.0822,NaN


In [28]:
test_set = test_set.merge(ant3,on = 'antiguedad',how = 'left')
test_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,59990,59991,59992,59993,59994,59995,59996,59997,59998,59999
id,4941,51775,115253,299321,173570,30862,244471,127794,71558,218011,...,205625,284266,70244,59776,79100,75094,171847,138313,271268,72612
antiguedad,29,NaN,0,2,10,10,20,0,2,20,...,NaN,5,6,NaN,NaN,20,10,5,0,0
habitaciones,3,1,2,2,2,3,3,2,3,3,...,3,2,3,NaN,NaN,4,3,3,2,3
garages,NaN,1,1,2,1,1,NaN,1,2,1,...,NaN,1,3,0,NaN,3,1,2,1,2
banos,4,1,2,2,1,2,2,1,2,2,...,3,2,2,NaN,NaN,3,2,2,2,2
metroscubiertos,300,67,87,86,80,165,220,62,260,123,...,316,68,270,NaN,199,291,71,102,130,211
metrostotales,NaN,67,100,86,76,138,190,62,200,160,...,NaN,68,170,200,199,NaN,87,NaN,144,130
idzona,NaN,113851.0,23620.0,129347.0,57125.0,48216.0,323485.0,54688.0,107969.0,50002836.0,...,55552.0,50003995.0,24940.0,49037.0,24597.0,275741.0,57474.0,72224.0,83960.0,47747.0
lat,19.4087,21.0325,19.3328,16.8605,19.6405,NaN,NaN,NaN,19.17,21.158,...,NaN,NaN,19.2812,20.5312,19.4409,19.4348,NaN,NaN,20.5918,20.656
lng,-99.2468,-89.5924,-99.1529,-99.8784,-99.1273,NaN,NaN,NaN,-96.1527,-86.8385,...,NaN,NaN,-99.1422,-103.487,-99.1885,-99.0925,NaN,NaN,-100.328,-103.427


### Feature: antiguedad por provincia

In [29]:
data.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239990,239991,239992,239993,239994,239995,239996,239997,239998,239999
id,254099,53461,247984,209067,185997,126147,139233,5013,44962,134537,...,87498,137337,54886,207892,110268,119879,259178,131932,146867,121958
titulo,depto. tipo a-402,condominio horizontal en venta,casa en venta urbi 3 recamaras tonala,casa sola en toluca zinacantepec con credito i...,paseos del sol,departamento en venta taxqueña,de oportunidad casa en san lorenzo,casa emilia en venta en selvamar playa del carmen,pre- venta preciosos depas 2 recamaras con sub...,terreno,...,casa en venta: bosques del contry,departamento residencial coyuya,casa en venta,bugambilias (ciudad),hermosa casa en villa de los belenes,bonita casas de 2 recamaras a 10 minutos del c...,casa en condominio a 10 min. del centro de toluca,nicolas san juan,casa sola. javier rojo gomez.,departamento en bosques de las lomas / av. st...
descripcion,"depto. interior de 80.15m2, consta de sala com...","<p>entre sonora y guerrero, atr&aacute;s del h...",descripcion \nla mejor ubicacion residencial e...,casa en privada con caseta de vigilancia casas...,bonito departamento en excelentes condiciones ...,"amplio departamento, estancia de sala y comedo...","ubicada en esquina, pertenece san lorenzo agen...",casa emilia en venta playa del carmenfracciona...,<p>pre-venta de preciosos departamento ecologi...,"terreno de 5.500m2 bardeado, uso de suelo h-20...",...,"<p>casa en venta, en magníficas condiciones; e...","departamento ubicado en planta baja, con excel...",bonita casa para remodelar en una calle cerrad...,coto privado de tan solo 7 casas donde cada fa...,"<p>moderna casa 3 pisos, muro llor&oacute;n , ...",vendo casa en bosques de ica residencial a 10 ...,"casa con un jardin amplio, un cuarto de servic...","departamento con excelente ubicación, muy cerc...","casa sola, dividida en cuatro departamentos de...","id:19816, muy bonito e iluminado departamento,..."
tipodepropiedad,Apartamento,Casa en condominio,Casa,Casa,Apartamento,Apartamento,Casa,Casa,Apartamento,Terreno,...,Casa,Apartamento,Casa en condominio,Casa,Casa,Casa,Casa,Apartamento,Casa,Apartamento
direccion,Avenida Division del Norte 2005,AV. MEXICO,Urbi Tonala,IGNACIO MANUEL ALTAMIRANO 128,PASEOS DEL SOL,Condominio Tlalpan 2B,NaN,condominio el trebol,BUENAVISTA DEPTOS CON SUBSIDIO,Av. Morelos,...,NaN,Coyuya 200,Cerrada villa Picadilly,Paseo de la Cañada,"MANUEL AMAYA,ENTRE SEBASTIAN ALLENDE Y AMADO A...",BOSQUES,Filiberto Navas 325,Nicolas San Juan,Javier Rojo Gomez 120,AVE. STIM
ciudad,Benito Juárez,La Magdalena Contreras,Tonalá,Zinacantepec,Zapopan,Coyoacán,Oaxaca de Juárez,Playa del Carmen,Villa de Alvarez,Ixtapaluca,...,Guadalupe,Iztacalco,Huixquilucan,Zapopan,Zapopan,Zinacantepec,Toluca,Benito Juárez,Iztapalapa,Cuajimalpa de Morelos
provincia,Distrito Federal,Distrito Federal,Jalisco,Edo. de México,Jalisco,Distrito Federal,Oaxaca,Quintana Roo,Colima,Edo. de México,...,Nuevo León,Distrito Federal,Edo. de México,Jalisco,Jalisco,Edo. de México,Edo. de México,Distrito Federal,Distrito Federal,Distrito Federal
antiguedad,NaN,10,5,1,10,5,NaN,2,1,NaN,...,20,20,10,1,3,0,0,20,20,1
habitaciones,2,3,3,2,2,2,3,4,2,NaN,...,3,2,3,3,2,2,3,2,4,3
garages,1,2,2,1,1,1,1,2,1,NaN,...,2,1,NaN,2,2,2,3,1,0,2


In [30]:
#Mergea el la columna indicada en 'on', tomando el df pasado como parámetro(df_to_use)
#how_feature indica que tipo de join se hace.
#El merge lo hace sobre 'train_set' y 'test_set'
def merge_df(df_to_use, on_feature,how_feature):
    print("dataframe a usar:"+str(df_to_use))
    print("feature a usar:"+on_feature)
   # print("[test_set] Cantidad init: "+str(len(test_set.columns)))
    #print("[train_set] Cantidad init: "+str(len(train_set.columns)))
    test_set.merge(df_to_use, on = on_feature, how = how_feature)
    train_set.merge(df_to_use, on = on_feature, how = how_feature)
    #print("[test_set] Cantidad init: "+str(len(test_set.columns)))
    #print("[train_set] Cantidad init: "+str(len(train_set.columns)))

In [31]:
df = data.groupby(by = 'provincia').agg({'antiguedad':'mean'}).reset_index()
df.sort_values(['antiguedad'], ascending = False, inplace = True)
df=df.rename(columns={"antiguedad":"antiguedad_prov"})
df.head()

data = data.merge(df,on='provincia',how='inner')
data.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239835,239836,239837,239838,239839,239840,239841,239842,239843,239844
id,254099,53461,126147,130216,73348,177031,202245,92610,116928,244353,...,163941,226592,275038,201744,194644,3654,190742,191824,57022,132311
titulo,depto. tipo a-402,condominio horizontal en venta,departamento en venta taxqueña,casa matias romero - sión bancaria,"oportunidad, departamento col del valle, 3 rec...",NaN,casa en venta en tlalpan,"casa en venta en agrícola oriental, iztacalco",excelentes departamentos con exclente ubicación,NaN,...,"casa habitacion en guadalupe, zac","edificio en residencial boulevares, zac. (ven...","excelente casa seminueva en fracc. la cañada, ...",casa en zacatecas seminueva fraccionamiento lo...,se vende hermosa casa en lomas del pedregal,casa en venta en calera,terreno en venta en zacatecas,casa en condominio en venta en guadalupe,"local y departamento en venta en sombrerete, zac.","casa en venta en san cosme, guadalupe"
descripcion,"depto. interior de 80.15m2, consta de sala com...","<p>entre sonora y guerrero, atr&aacute;s del h...","amplio departamento, estancia de sala y comedo...",flamante casa habitación en condominio horizon...,magnifico departamento con excelente distribuc...,"hermosa casa remodelada, con acabados de lujo,...","hermosa residencia con acabados de lujo, ¡opor...",<p>casa duplex (el primer nivel es el que se e...,"1 pb, 2 primer piso, 2 segundo piso. cada depa...","departamento nuevo, 98m2 ,interior, 2 recamara...",...,"casa habitacion en privada, de una planta y co...",<table cellspacing=0 cellpadding=0 align=left>...,"casa seminueva, con buenos acabados y amplios ...","sala (doble altura), comedor, cocina, 3 recama...","casa con diseño minimalista, nueva en fraccion...",nocnok id: mx14-aw0878. la vivienda se encuent...,"<p>terreno en venta en zacatecas, ubicado en ...","casa dúplex planta baja en una esquina, 3 habi...",<table cellspacing=0 cellpadding=0 align=left>...,"<p>lote 6 y 16 del fracc. san cosme, guadalupe..."
tipodepropiedad,Apartamento,Casa en condominio,Apartamento,Casa,Apartamento,Casa en condominio,Casa,Casa,Apartamento,Apartamento,...,Casa,Edificio,Casa en condominio,Casa,Casa,Casa,Terreno,Casa en condominio,Edificio,Casa
direccion,Avenida Division del Norte 2005,AV. MEXICO,Condominio Tlalpan 2B,"MATIAS ROMERO, COLONIA DEL VALLE",Pazaje Santa Cruz,PASEO DE LOS LAURELES,Galeana,Sur 16,Jesús Carranza No. 55,ILLINOIS 31 602,...,mina valenciana,"CALLE UNO, BOULEVARES, ZACATECAS, ZAC.","Roca, Fracc. La Cañada 59",Cerro del Ángel 22,Circuito del Pedregal #94,NaN,NaN,Andador Capulines 1 A,"JARDIN HIDALGO #204, ZONA CENTRO, SOMBRERETE, ...",LOTE 6 Y 16
ciudad,Benito Juárez,La Magdalena Contreras,Coyoacán,Benito Juárez,Benito Juárez,Miguel Hidalgo,Tlalpan,Iztacalco,Cuauhtémoc,Benito Juárez,...,Guadalupe,Zacatecas,Guadalupe,Guadalupe,Guadalupe,Calera,Jerez,Guadalupe,Sombrerete,Guadalupe
provincia,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,...,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas
antiguedad,NaN,10,5,20,5,18,2,16,20,0,...,0,10,3,3,2,NaN,6,20,10,10
habitaciones,2,3,2,NaN,3,3,3,3,2,2,...,2,NaN,3,3,3,3,NaN,3,NaN,4
garages,1,2,1,0,2,3,2,1,0,2,...,1,0,1,1,2,NaN,NaN,1,0,0


In [32]:
train_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,anio_publ,c_hab_ant
0,254099,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2273000.0,2015,NaN
1,53461,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,19.310205,-99.227655,0.0,0.0,0.0,1.0,1.0,3600000.0,2013,28844.0
2,247984,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1200000.0,2015,33268.0
3,209067,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,19.301890,-99.688015,0.0,0.0,0.0,1.0,1.0,650000.0,2012,12353.0
4,185997,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,28844.0


In [33]:
train_set = train_set.merge(data[['id','antiguedad_prov']], on = 'id', how = 'left')
test_set = test_set.merge(data[['id','antiguedad_prov']], on = 'id', how = 'left')

train_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239990,239991,239992,239993,239994,239995,239996,239997,239998,239999
id,254099,53461,247984,209067,185997,126147,139233,5013,44962,134537,...,87498,137337,54886,207892,110268,119879,259178,131932,146867,121958
antiguedad,NaN,10,5,1,10,5,NaN,2,1,NaN,...,20,20,10,1,3,0,0,20,20,1
habitaciones,2,3,3,2,2,2,3,4,2,NaN,...,3,2,3,3,2,2,3,2,4,3
garages,1,2,2,1,1,1,1,2,1,NaN,...,2,1,NaN,2,2,2,3,1,0,2
banos,2,2,2,1,1,1,2,3,1,NaN,...,1,1,3,3,2,1,3,2,4,2
metroscubiertos,80,268,144,63,95,75,140,293,58,250,...,118,68,270,150,160,67,200,138,235,149
metrostotales,80,180,166,67,95,90,160,293,NaN,NaN,...,124,68,293,150,90,NaN,250,138,137,NaN
idzona,23533.0,24514.0,48551.0,53666.0,47835.0,23650.0,73510.0,130510.0,9010.0,59171.0,...,72083.0,24118.0,55589.0,48046.0,48029.0,53666.0,51954.0,50003995.0,24162.0,23750.0
lat,NaN,19.3102,NaN,19.3019,NaN,19.3006,17.1435,20.6726,NaN,19.316,...,25.6612,19.4033,NaN,20.6156,20.6045,NaN,19.2947,NaN,19.3667,NaN
lng,NaN,-99.2277,NaN,-99.688,NaN,-99.1485,-96.8035,-87.038,NaN,-98.887,...,-100.251,-99.119,NaN,-103.456,-103.45,NaN,-99.6929,NaN,-99.0822,NaN


In [34]:
test_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,59990,59991,59992,59993,59994,59995,59996,59997,59998,59999
id,4941,51775,115253,299321,173570,30862,244471,127794,71558,218011,...,205625,284266,70244,59776,79100,75094,171847,138313,271268,72612
antiguedad,29,NaN,0,2,10,10,20,0,2,20,...,NaN,5,6,NaN,NaN,20,10,5,0,0
habitaciones,3,1,2,2,2,3,3,2,3,3,...,3,2,3,NaN,NaN,4,3,3,2,3
garages,NaN,1,1,2,1,1,NaN,1,2,1,...,NaN,1,3,0,NaN,3,1,2,1,2
banos,4,1,2,2,1,2,2,1,2,2,...,3,2,2,NaN,NaN,3,2,2,2,2
metroscubiertos,300,67,87,86,80,165,220,62,260,123,...,316,68,270,NaN,199,291,71,102,130,211
metrostotales,NaN,67,100,86,76,138,190,62,200,160,...,NaN,68,170,200,199,NaN,87,NaN,144,130
idzona,NaN,113851.0,23620.0,129347.0,57125.0,48216.0,323485.0,54688.0,107969.0,50002836.0,...,55552.0,50003995.0,24940.0,49037.0,24597.0,275741.0,57474.0,72224.0,83960.0,47747.0
lat,19.4087,21.0325,19.3328,16.8605,19.6405,NaN,NaN,NaN,19.17,21.158,...,NaN,NaN,19.2812,20.5312,19.4409,19.4348,NaN,NaN,20.5918,20.656
lng,-99.2468,-89.5924,-99.1529,-99.8784,-99.1273,NaN,NaN,NaN,-96.1527,-86.8385,...,NaN,NaN,-99.1422,-103.487,-99.1885,-99.0925,NaN,NaN,-100.328,-103.427


# Se exporta para entrenar

## NO hace falta eliminar NULLs

In [35]:
len(train_set)

240000

In [36]:
train_set.to_csv("train_set_xgb.csv", index = False)

In [37]:
test_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,anio_publ,c_hab_ant,antiguedad_prov
0,4941,29.0,3.0,NaN,4.0,300.0,NaN,NaN,19.408668,-99.246767,0.0,0.0,0.0,0.0,0.0,2013,196.0,NaN
1,51775,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,21.032480,-89.592424,0.0,0.0,0.0,0.0,0.0,2015,NaN,NaN
2,115253,0.0,2.0,1.0,2.0,87.0,100.0,23620.0,19.332829,-99.152913,0.0,0.0,0.0,0.0,1.0,2015,50335.0,NaN
3,299321,2.0,2.0,2.0,2.0,86.0,86.0,129347.0,16.860487,-99.878383,0.0,0.0,0.0,0.0,0.0,2015,5059.0,NaN
4,173570,10.0,2.0,1.0,1.0,80.0,76.0,57125.0,19.640482,-99.127273,0.0,0.0,0.0,1.0,1.0,2013,28844.0,NaN


In [38]:
len(test_set)

60000

In [39]:
test_set.to_csv("test_set_xgb.csv", index = False)